# This week, I used a small dataset to test my toy deep learning + survival analysis model.

## Preparation

In [1]:
from keras.models import Sequential
import keras.backend as K
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.regularizers import l2
from keras.optimizers import Adagrad
from lifelines.utils import concordance_index
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os, sys

Using TensorFlow backend.


The following loss function is adapted from DeepSurv: Personalized Treatment Recommender System Using A Cox Proportional Hazards Deep Neural Network.
<cite>Katzman J L, Shaham U, Cloninger A, et al. DeepSurv: personalized treatment recommender system using a Cox proportional hazards deep neural network[J]. Bmc Medical Research Methodology, 2018, 18(1):24.</cite>

In [2]:
def negative_log_likelihood(E):
    def loss(y_true,y_pred):
        hazard_ratio = K.exp(y_pred)
        log_risk = K.log(K.cumsum(hazard_ratio))
        uncensored_likelihood = K.transpose(y_pred) - log_risk
        censored_likelihood = uncensored_likelihood * E
        num_observed_event = K.sum([float(e) for e in E])
        return K.sum(censored_likelihood) / num_observed_event * (-1)
    return loss

Create a simple and sallow model to test whether a deep learning model can integrate with the Cox proportional-hazards model. 

In [3]:
def gen_model(input_shape, size=32):
    model = Sequential()
    model.add(Dense(size, activation='relu', kernel_initializer='glorot_uniform',
    input_shape=input_shape))
    model.add(Dense(size, activation='relu', kernel_initializer='glorot_uniform'))
    model.add(Dropout(0.8))
    # model.add(Dense(128, activation='relu', kernel_initializer='glorot_uniform'))
    # model.add(Dropout(0.5))
    model.add(Dense(1, activation="linear", kernel_initializer='glorot_uniform', 
    kernel_regularizer=l2(0.01), activity_regularizer=l2(0.01)))

    return model

Process the manual created data to the form which fits the learning network above. The data have been preprocessed by the same way as the experiment "baseline_establishment". Aa a result, it's from the "dummy.xlsx" that experiment creaeted.

In [4]:
def gen_data(fp):
    df = pd.read_excel(fp)
    y = (df.loc[:, ["duration","observed"]]).values
    x = (df.loc[:,'epi_area_<25% (Low)':]).values

    return x, y

In [5]:
x, y = gen_data('dummy.xlsx')

In [6]:
print(f'{x.shape[0]} samples are here. Each sample has {x.shape[1]} features')

760 samples are here. Each sample has 21 features


In [7]:
toy_model = gen_model(x.shape[1:])
toy_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 32)                704       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,793
Trainable params: 1,793
Non-trainable params: 0
_________________________________________________________________


Again, I use  concordance index (CI) to judge the performance of the model. The concordance index (CI), which quantifies the quality of rankings, is the standard performance measure for model assessment in survival analysis.

In [8]:
def eval(model, x, y, e):
    hr_pred=model.predict(x)
    hr_pred=np.exp(hr_pred)
    ci=concordance_index(y,-hr_pred,e)
    return ci

## Ready to Run

Caveat! Usually, deep learning models will shuffle the data, that is to change the order of data randomly. In doing so, the models can perform well in different circumstances. However, for survival analysis, many data are censored, i.e. the event didn't happen in some cases. To make these censored data useful, all data should keep in a chronological order, or the model cannot be trained successfully.

In order to repeat it conveniently, I combined these functions into a monolithic one.

In [9]:
def run_model(fp, dst, size=32):
    x, t_e = gen_data(fp)
    x_train, x_test, t_e_train, t_e_test = train_test_split(x, t_e, test_size=0.2, 
                                                            random_state=42)
    # print(t_e_train[:10])
    y_train, e_train = t_e_train[:, 0], t_e_train[:, 1]
    y_test, e_test = t_e_test[:, 0], t_e_test[:, 1]

    sort_idx = np.argsort(y_train)[::-1] #! chronological order
    x_train = x_train[sort_idx]
    y_train = y_train[sort_idx]
    e_train = e_train[sort_idx]

    x_t_shape = np.shape(x_train)
    print('{} training images have prepared, shape is {}\
    and {}'.format(len(x_train), x_t_shape, np.shape(y_train)))
    print('{} test images have prepared, shape is {}\
    and {}'.format(len(x_test), np.shape(x_test), np.shape(y_test)))

    model = gen_model(x_t_shape[1:], size)
    model.summary()
    ada = Adagrad(lr=1e-3, decay=0.1)
    model.compile(loss=negative_log_likelihood(e_train), optimizer=ada)

    cheak_list = [EarlyStopping(monitor='loss', patience=10),
                ModelCheckpoint(filepath=os.path.join(dst, 'toy.h5')
                , monitor='loss', save_best_only=True),
                TensorBoard(log_dir=os.path.join(dst, 'toy_log'), 
                histogram_freq=0)]

    model.fit(
        x_train, y_train,
        batch_size=len(e_train),
        epochs=100,
        verbose=True,
        callbacks=cheak_list,
        shuffle=False)
    
    ci = eval(model, x_train, y_train, e_train)
    ci_val = eval(model, x_test, y_test, e_test)
    
    with open(os.path.join(dst, 'toy_outcome.txt'), 'w+') as out:
        line = 'Concordance Index for training dataset:{},\n\
                Concordance Index for test dataset:{}'.format(ci, ci_val)
        print(line)
        out.write(line)

## Run and Results

In [10]:
fp = 'dummy.xlsx'
dst = 'toy_experiment'
os.makedirs(dst, exist_ok=True)
run_model(fp, dst, size=32)

608 training images have prepared, shape is (608, 21)    and (608,)
152 test images have prepared, shape is (152, 21)    and (152,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 32)                704       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 1,793
Trainable params: 1,793
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
608/608 [==============================] - 1s 972us/step - loss: 3430.7344
Epoch 2/100
608/608 [==============================] 

The model can run correctly. However, Concordance Indexes for training dataset and for test dataset are both pretty low. It's possibly due to the limited capacity of the model. So I tried to increace its size.

In [11]:
run_model(fp, dst, size=256)

608 training images have prepared, shape is (608, 21)    and (608,)
152 test images have prepared, shape is (152, 21)    and (152,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 256)               5632      
_________________________________________________________________
dense_8 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 71,681
Trainable params: 71,681
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
608/608 [==============================] - 1s 1ms/step - loss: 3290.6594
Epoch 2/100
608/608 [==============================] 

We see the outcome become better now. Still the Concordance Index for test dataset is low. That implies the capacity of the model is too powerful to generalize varing data. In this case, we should shrink it.

In [12]:
run_model(fp, dst, size=128)

608 training images have prepared, shape is (608, 21)    and (608,)
152 test images have prepared, shape is (152, 21)    and (152,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 128)               2816      
_________________________________________________________________
dense_11 (Dense)             (None, 128)               16512     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 19,457
Trainable params: 19,457
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
608/608 [==============================] - 1s 1ms/step - loss: 3367.4153
Epoch 2/100
608/608 [==============================] 

Since the population of samples is small, even though the model could learn from the data, and the outcome followed the basic rules of deep learning model, the result was not desired. I should do more research nextweek.